In [ ]:
!pip3 install pickle5
!pip install pyLDAvis
import pickle5 as pickle
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
from gensim.models.ldamodel import LdaModel

In [ ]:
data = pickle.load(open('data_hgr_preprocessed_2020.pkl', "rb" )) #데이터 불러오기
data['month']=data['datetime'].map(lambda x : x.month)

In [ ]:
def lda (body):
  dictionary = corpora.Dictionary(body)
  dictionary.filter_extremes(no_below=10)
  corpus = [dictionary.doc2bow(text) for text in body]
  NUM_TOPICS =5
  lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=5)
  topics = lda_model.print_topics(num_words=20)
  return lda_model, topics, corpus

In [ ]:
lda_model_whole, topic_whole, corpus_whole =lda(data['body'])

In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()


    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)

        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.



        # 모든 문서에 대해서 각각 아래를 수행

        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.

            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽

                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)

                # 가장 비중이 높은 토픽 & 비중, 전체 토픽의 비중을 저장한다.

            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(lda_model_whole, corpus_whole)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable

In [ ]:
topictable['month']=data['month'].reset_index()['month']

In [ ]:
topic_words=dict(lda_model_whole.print_topics())
topic_words

In [ ]:
topic=[0,2,3]
montly_top_list=[]
def get_ratio(data):
  for a in range(1,13):
    df=data[data['month']==a]
    for i in topic:
      print(a,"월에",i, "번 째 토픽 기사개수는", len(df[df['가장 비중이 높은 토픽']==i]))
      k=len(df[df['가장 비중이 높은 토픽']==i])
      montly_top_list.append([a,i,k])
get_ratio(topictable)